In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Lavrentiya,RU,65.5833,-171.000,-13.34,84,0,4.85,clear sky
1,1,Hilo,US,19.7297,-155.090,78.80,61,20,8.05,few clouds
2,2,Kapaa,US,22.0752,-159.319,77.00,69,1,2.82,clear sky
3,3,Shimoda,JP,34.6667,138.950,41.00,52,75,16.11,broken clouds
4,4,Dukat,RU,62.5500,155.550,-15.20,82,1,1.21,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Hilo,US,19.7297,-155.0900,78.80,61,20,8.05,few clouds
2,2,Kapaa,US,22.0752,-159.3190,77.00,69,1,2.82,clear sky
5,5,Mahebourg,MU,-20.4081,57.7000,77.00,78,40,9.22,scattered clouds
6,6,Hobart,AU,-42.8794,147.3294,81.00,56,20,17.27,few clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,83.28,79,99,16.62,overcast clouds
12,12,Palasa,IN,18.7667,84.4167,80.19,43,4,2.17,clear sky
19,19,Arraial Do Cabo,BR,-22.9661,-42.0278,75.58,92,68,12.77,broken clouds
20,20,Georgetown,MY,5.4112,100.3354,86.00,74,20,6.91,few clouds
21,21,Kulhudhuffushi,MV,6.6221,73.0700,82.47,74,73,8.66,broken clouds
25,25,Bengkulu,ID,-3.8004,102.2655,88.21,54,81,4.12,moderate rain


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                162
City                   162
Country                161
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Weather Description    162
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Hilo,US,19.7297,-155.0900,78.80,61,20,8.05,few clouds
2,2,Kapaa,US,22.0752,-159.3190,77.00,69,1,2.82,clear sky
5,5,Mahebourg,MU,-20.4081,57.7000,77.00,78,40,9.22,scattered clouds
6,6,Hobart,AU,-42.8794,147.3294,81.00,56,20,17.27,few clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,83.28,79,99,16.62,overcast clouds


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Hilo,US,78.80,few clouds,19.7297,-155.0900,
2,Kapaa,US,77.00,clear sky,22.0752,-159.3190,
5,Mahebourg,MU,77.00,scattered clouds,-20.4081,57.7000,
6,Hobart,AU,81.00,few clouds,-42.8794,147.3294,
9,Hithadhoo,MV,83.28,overcast clouds,-0.6000,73.0833,
12,Palasa,IN,80.19,clear sky,18.7667,84.4167,
19,Arraial Do Cabo,BR,75.58,broken clouds,-22.9661,-42.0278,
20,Georgetown,MY,86.00,few clouds,5.4112,100.3354,
21,Kulhudhuffushi,MV,82.47,broken clouds,6.6221,73.0700,
25,Bengkulu,ID,88.21,moderate rain,-3.8004,102.2655,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Hilo,US,78.80,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Kapaa,US,77.00,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
5,Mahebourg,MU,77.00,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Hobart,AU,81.00,few clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
9,Hithadhoo,MV,83.28,overcast clouds,-0.6000,73.0833,Scoop Guest House


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Hilo,US,78.80,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Kapaa,US,77.00,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
5,Mahebourg,MU,77.00,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Hobart,AU,81.00,few clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
9,Hithadhoo,MV,83.28,overcast clouds,-0.6000,73.0833,Scoop Guest House


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))